# pytorch faster rcnn

In [3]:
!apt install time
!pip install tensorboardX
!pip install scipy==1.1.0

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-410
Use 'apt autoremove' to remove it.
The following NEW packages will be installed:
  time
0 upgraded, 1 newly installed, 0 to remove and 8 not upgraded.
Need to get 26.2 kB of archives.
After this operation, 79.9 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/main amd64 time amd64 1.7-25.1build1 [26.2 kB]
Fetched 26.2 kB in 0s (185 kB/s)
Selecting previously unselected package time.
(Reading database ... 130912 files and directories currently installed.)
Preparing to unpack .../time_1.7-25.1build1_amd64.deb ...
Unpacking time (1.7-25.1build1) ...
Setting up time (1.7-25.1build1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
     |████████████████████████████████| 245kB 9.7MB/s 


In [4]:
import torch
torch.__version__

'1.1.0'

In [5]:
!wget http://host.robots.ox.ac.uk/pascal/VOC/voc2007/VOCtrainval_06-Nov-2007.tar
!wget http://host.robots.ox.ac.uk/pascal/VOC/voc2007/VOCtest_06-Nov-2007.tar
!wget http://host.robots.ox.ac.uk/pascal/VOC/voc2007/VOCdevkit_08-Jun-2007.tar

--2019-06-15 09:30:24--  http://host.robots.ox.ac.uk/pascal/VOC/voc2007/VOCtrainval_06-Nov-2007.tar
Resolving host.robots.ox.ac.uk (host.robots.ox.ac.uk)... 129.67.94.152
Connecting to host.robots.ox.ac.uk (host.robots.ox.ac.uk)|129.67.94.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 460032000 (439M) [application/x-tar]
Saving to: ‘VOCtrainval_06-Nov-2007.tar’

VOCtrainval_06-Nov- 100%[===================>] 438.72M  57.1MB/s    in 7.1s    

2019-06-15 09:30:31 (61.5 MB/s) - ‘VOCtrainval_06-Nov-2007.tar’ saved [460032000/460032000]

--2019-06-15 09:30:33--  http://host.robots.ox.ac.uk/pascal/VOC/voc2007/VOCtest_06-Nov-2007.tar
Resolving host.robots.ox.ac.uk (host.robots.ox.ac.uk)... 129.67.94.152
Connecting to host.robots.ox.ac.uk (host.robots.ox.ac.uk)|129.67.94.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 451020800 (430M) [application/x-tar]
Saving to: ‘VOCtest_06-Nov-2007.tar’

VOCtest_06-Nov-2007 100%[===================

In [6]:
!git clone https://github.com/ruotianluo/pytorch-faster-rcnn

Cloning into 'pytorch-faster-rcnn'...
remote: Enumerating objects: 27, done.
remote: Counting objects: 100% (27/27), done.
remote: Compressing objects: 100% (27/27), done.
remote: Total 2332 (delta 8), reused 1 (delta 0), pack-reused 2305
Receiving objects: 100% (2332/2332), 21.89 MiB | 18.76 MiB/s, done.
Resolving deltas: 100% (1566/1566), done.


In [0]:
import os

In [0]:
os.chdir('./pytorch-faster-rcnn/')

In [0]:
!cd data/; tar xf /content/VOCdevkit_08-Jun-2007.tar; tar xf /content/VOCtest_06-Nov-2007.tar; tar xf /content/VOCtrainval_06-Nov-2007.tar

In [10]:
!ls data/

demo  imgs  scripts  VOCdevkit


In [0]:
!cd data; ln -s /content/pytorch-faster-rcnn/data/VOCdevkit/ VOCdevkit2007;

In [12]:
!ls -lht ./data

total 16K
lrwxrwxrwx 1 root root   44 Jun 15 09:30 VOCdevkit2007 -> /content/pytorch-faster-rcnn/data/VOCdevkit/
drwxr-xr-x 2 root root 4.0K Jun 15 09:30 scripts
drwxr-xr-x 2 root root 4.0K Jun 15 09:30 imgs
drwxr-xr-x 2 root root 4.0K Jun 15 09:30 demo
drwxrwxrwx 6 root root 4.0K Nov  6  2007 VOCdevkit


In [0]:
!mkdir -p data/imagenet_weights
!cd data/imagenet_weights

In [14]:
!ls

data  docker  experiments  lib	LICENSE  README.md  tools


In [0]:
os.chdir('data/imagenet_weights/')

In [16]:
!pwd

/content/pytorch-faster-rcnn/data/imagenet_weights


In [17]:
# download pre-trained vgg16 weights and save to $pytorch-faster-rcnn/data/i
import torch
from torch.utils.model_zoo import load_url
from torchvision import models

sd = load_url("https://s3-us-west-2.amazonaws.com/jcjohns-models/vgg16-00b39a1b.pth")
sd['classifier.0.weight'] = sd['classifier.1.weight']
sd['classifier.0.bias'] = sd['classifier.1.bias']
del sd['classifier.1.weight']
del sd['classifier.1.bias']

sd['classifier.3.weight'] = sd['classifier.4.weight']
sd['classifier.3.bias'] = sd['classifier.4.bias']
del sd['classifier.4.weight']
del sd['classifier.4.bias']

torch.save(sd, "vgg16.pth")

Downloading: "https://s3-us-west-2.amazonaws.com/jcjohns-models/vgg16-00b39a1b.pth" to /root/.cache/torch/checkpoints/vgg16-00b39a1b.pth
100%|██████████| 553451520/553451520 [00:27<00:00, 19786523.57it/s]


In [18]:
!ls -lht

total 528M
-rw-r--r-- 1 root root 528M Jun 15 09:31 vgg16.pth


In [19]:
os.chdir('../../')
!pwd

/content/pytorch-faster-rcnn


In [20]:
!./experiments/scripts/train_faster_rcnn.sh 0 pascal_voc vgg16

+ set -e
+ export PYTHONUNBUFFERED=True
+ PYTHONUNBUFFERED=True
+ GPU_ID=0
+ DATASET=pascal_voc
+ NET=vgg16
+ array=($@)
+ len=3
+ EXTRA_ARGS=
+ EXTRA_ARGS_SLUG=
+ case ${DATASET} in
+ TRAIN_IMDB=voc_2007_trainval
+ TEST_IMDB=voc_2007_test
+ STEPSIZE='[50000]'
+ ITERS=70000
+ ANCHORS='[8,16,32]'
+ RATIOS='[0.5,1,2]'
++ date +%Y-%m-%d_%H-%M-%S
+ LOG=experiments/logs/vgg16_voc_2007_trainval__vgg16.txt.2019-06-15_09-31-58
+ exec
++ tee -a experiments/logs/vgg16_voc_2007_trainval__vgg16.txt.2019-06-15_09-31-58
+ echo Logging output to experiments/logs/vgg16_voc_2007_trainval__vgg16.txt.2019-06-15_09-31-58
Logging output to experiments/logs/vgg16_voc_2007_trainval__vgg16.txt.2019-06-15_09-31-58
+ set +x
+ '[' '!' -f output/vgg16/voc_2007_trainval/default/vgg16_faster_rcnn_iter_70000.pth.index ']'
+ [[ ! -z '' ]]
+ CUDA_VISIBLE_DEVICES=0
+ time python ./tools/trainval_net.py --weight data/imagenet_weights/vgg16.pth --imdb voc_2007_trainval --imdbval voc_2007_test --iters 70000 --cfg experime

In [0]:
# from skimage.transform import rescale, resize, downscale_local_mean